# Introduction to TenSEAL

TenSEAL is a tensor library, just like PyTorch or Tensorflow, however, instead of manipulating tensors that holds plain data, tensors in TenSEAL holds encrypted data using mainly the CKKS scheme.

In [1]:
import tenseal as ts

## The Context

The first object you want to create whenever you deal with TenSEAL is the context. It provides ways to control how the computation is performed, generate and holds the keys (e.g. the secret-key and public-key) required during encryption and computation.

In [2]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])

The above line created a context for CKKS encryption, using those specific parameters. We will provide some intuition on how to choose those parameters shortly. The context creation above took care of generating keys required, and you can now get the secret-key for example.

In [3]:
sk = context.secret_key()
sk

## Intuitions for Parameter Selection in CKKS

It's hard to choose the right parameters for a certain computation, but if you have some intuitions of what every parameter might be affecting, you can make some experiments using some different variants and see which works best for your use case. Below are some intuitions for choosing the parameters:

- For a given security level (e.g. 128-bits security) and a polynomial modulus degree (e.g. 8192) there is an upper bound for the `sum(coeff_mod_bit_sizes)`. If the upper bound is surpassed, there is a need to use a higher polynomial modulus degree (e.g. 16384) in order to make sure we still have the required security level.
- The multiplicative depth is controlled by `len(coeff_mod_bit_sizes) - 2`. The above context provide a multiplicative depth of 2 for example. A `coeff_mod_bit_sizes=[60, 40, 40, 40, 60]` would have provided 3.
- All elements of `coeff_mod_bit_sizes[1: -1]` should be equal in TenSEAL, since it takes care of rescaling ciphertexts. And we also want to use this same number of bits (e.g. 2 ^ 40) for the scale during encryption.
- The scale of ciphertext (which should be equal to `coeff_mod_bit_sizes[1: -1]`) controls the precision of the fractional part. The bigger, the more precise.
- The difference between `coeff_mod_bit_sizes[-1]` and `coeff_mod_bit_sizes[-2]` (or the scale) controls the precision in the integer part when all multiplication have been consumed. The bigger, the more precise.

## Creating Tensors

Now we start creating some encrypted tensors. You can pass in any tensor-like object (it has been tested with torch and numpy), and TenSEAL would take care of encrypting it using the keys and parameters stored in the context.

In [4]:
import numpy as np

plain_tensor = np.random.randn(2, 3)
encrypted_tensor = ts.ckks_tensor(context, plain_tensor, scale=2**40)
encrypted_tensor

You can also set the `global_scale` in the context so that you never need to pass the scale again and again during tensor creation

In [5]:
context.global_scale = 2 ** 40
encrypted_tensor = ts.ckks_tensor(context, plain_tensor)
encrypted_tensor

In [6]:
print(encrypted_tensor.decrypt())

The PlainTensor object is a tensor that we mainly use internally to represent tensors with plain values. You can always call `tolist()` to convert it to a list.

In [7]:
print(encrypted_tensor.decrypt().tolist())

[[-0.6935322486262878, -0.1971640039349987, -0.6806255185457504], [-0.6699472410544948, 1.3804208496159256, -0.05087860773279334]]


In [8]:
# you can compare it to the plain tensor
print(plain_tensor)

[[-0.69353225 -0.197164   -0.68062552]
 [-0.66994724  1.38042085 -0.05087861]]


## Compute on Encrypted Tensors

CKKSTensor support addition, subtraction, and multiplication with scalar and tensor-like values. Let's see an example computation.

In [9]:
encrypted_result = (encrypted_tensor + 2) * -3 - plain_tensor
expected_result = (plain_tensor + 2) * -3 - plain_tensor

In [10]:
encrypted_result.decrypt().tolist()

[[-3.2258715329047387, -5.211344710933246, -3.277498460263318],
 [-3.320211576555451, -11.521684756232458, -5.796486356321944]]

In [11]:
print(expected_result)

[[ -3.22587101  -5.21134399  -3.27749793]
 [ -3.32021105 -11.52168339  -5.79648557]]


We can also make dot operations!

In [12]:
# inner product
vec1 = np.random.randn(5)
vec2 = np.random.randn(5)
enc_vec1 = ts.ckks_tensor(context, vec1)
enc_vec2 = ts.ckks_tensor(context, vec2)
print("result:", enc_vec1.dot(enc_vec2).decrypt().tolist())
print("expected:", vec1.dot(vec2))

result: 0.2651245105129444
expected: 0.2651244888032714


In [13]:
# matrix multiplication
mat1 = np.random.randn(2, 3)
mat2 = np.random.randn(3, 4)
enc_mat1 = ts.ckks_tensor(context, mat1)
enc_mat2 = ts.ckks_tensor(context, mat2)
print("result:", enc_mat1.dot(enc_mat2).decrypt().tolist())
print("expected:", mat1.dot(mat2))

result: [[1.073233344180927, -1.244239415938116, -3.551577053659822, 0.1420384564803854], [0.07071465417811199, 2.8233036003449787, 3.6074107515157654, 0.38663331431247283]]
expected: [[ 1.0732332  -1.24423924 -3.55157659  0.14203844]
 [ 0.07071466  2.82330321  3.60741026  0.38663327]]


### Batched computation

We previously discussed in the CKKS concept that a single ciphertext can hold multiple values (a vector of values), without affecting the computation or memory costs. However, we didn't use that feature so far. So now, we are going to use this capability to compute on a batch of matrices at the same time.

In [14]:
# a single ciphertext can hold up to `poly_modulus_degree / 2` values
# so let's use all the slots available
batch_size = 8192 // 2 #  4096
mat1 = np.random.randn(batch_size, 2, 3)
mat2 = np.random.randn(3, 4)
# batch is by default set to False, we have to turn it on to use the packing feature of ciphertexts
enc_mat1 = ts.ckks_tensor(context, mat1, batch=True)
enc_mat2 = ts.ckks_tensor(context, mat2)
# let's just compare the first result matrix in the batch
print("result:", enc_mat1.dot(enc_mat2).decrypt().tolist()[0])
print("expected:", mat1.dot(mat2)[0])

result: [[1.8717073107411415, -1.646466405990874, 0.9077041758823494, 1.0415948014188408], [-1.5680046326356467, 1.127137122149572, -1.6876492182683538, -0.5806471841318446]]
expected: [[ 1.87170707 -1.64646619  0.90770405  1.04159466]
 [-1.56800442  1.12713697 -1.68764899 -0.58064711]]


## More Details

This section provides more details about the library and how some things happens under the hood.

### Parallel Computation

TenSEAL takes advantage of parallel computation whenever possible. It uses by default the maximum number of threads the system can run in parallel, but someone can specify that during context creation.

In [15]:
non_parallel_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
    n_threads=1,
)

### Decryption

you might have been wondering how is decryption not requiring a secret-key? But it's just because the context by default holds all the keys, and is considered a private asset. If you want to make it public, then you just call `make_context_public()`. Make sure you save the secret-key somewhere before that to use it later for decryption.

In [16]:
sk = context.secret_key()
context.make_context_public()

In [17]:
# now let's try decryption
enc_mat1.decrypt()

ValueError: the current context of the tensor doesn't hold a secret_key, please provide one as argument

In [18]:
# now we need to explicitly pass the secret-key
enc_mat1.decrypt(sk)

You should always make a context public before sending it to other parties to compute on encrypted data.

### Serialization

Speaking of sending the context and encrypted data, can we serialize these objects? And the answer is yes! There is a `serialize` method for every serializable object, like the context and tensors. Although, tensors have a slightly different ways of being loaded, let's see through an example.

In [19]:
ser_context = context.serialize()
type(ser_context)

bytes

In [20]:
ser_tensor = encrypted_tensor.serialize()
type(ser_tensor)

bytes

In [21]:
loaded_context = ts.context_from(ser_context)
loaded_context

As tensors always require to be linked to a context, we need to know which context to link the tensor to during deserialization

In [22]:
loaded_enc_tensor = ts.ckks_tensor_from(loaded_context, ser_tensor)
loaded_enc_tensor

However, there is also a way to do it the lazy way, deserializing, then linking it to a specific context

In [23]:
lazy_loaded_enc_tensor = ts.lazy_ckks_tensor_from(ser_tensor)
# try to operate on a tensor that in not linked to a context yet
lazy_loaded_enc_tensor + 5

ValueError: missing context

In [24]:
# You have to first link it
lazy_loaded_enc_tensor.link_context(loaded_context)
lazy_loaded_enc_tensor + 5

Now we can plug all these features with a system that can pass data between parties and we can do encrypted machine learning inference, where one party encrypts its data and send it to another party that do the inference while data stays encrypted, then send back the encrypted result. The initial party can then use the secret-key to decrypt the result. And fortunately, we already have Duet in PySyft to do this!